In [ ]:
!pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 8.5 MB/s eta 0:00:00


In [ ]:
#importing required libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import r2_score
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
import lightgbm
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet, ElasticNetCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.model_selection import StratifiedKFold, train_test_split, cross_val_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import r2_score

In [ ]:
#imporing dataset
df = pd.read_csv('/content/drive/MyDrive/python 2023/weaving_rejection_dataset - original.csv')

In [ ]:
df.head()

,Construction,Req_Finish_Fabrics,Fabric_Allowance,Rec_Beam_length(yds),Shrink_allow,Previous_pdn,Req_grey_fabric,Req_beam_length(yds),Total_Pdn(yds),Rejection,warp_count,weft_count,epi,ppi
0,40+40/2/40/110x80,31300.0,6.0,38286.9360,12.5,TOTAL,33297.872340,34797.651103,27646.0,285,double_40,80.0,110,80
1,40x40/110x90,10450.0,7.0,13057.5840,14.5,TOTAL,11236.559140,12017.350432,11019.0,39,40,40.0,110,90
2,40x40/110x80,900.0,11.5,1902.8640,12.5,TOTAL,1016.949153,1062.753843,1528.0,0,40,40.0,110,80
3,40x40/130x80,8000.0,8.0,10093.9280,14.0,TOTAL,8695.652174,9245.815159,8944.0,58,40,40.0,130,80
4,50x50/140x70,3500.0,8.1,69146.1408,9.1,TOTAL,3808.487486,3831.158751,59367.0,1043,50,50.0,140,70


In [ ]:
#selecting the required columns
df= df[['Req_Finish_Fabrics', 'Fabric_Allowance','Rec_Beam_length(yds)', 'Shrink_allow', 'Req_grey_fabric',
    'Req_beam_length(yds)', 'warp_count',  'weft_count','epi', 'ppi', 'Rejection']]

In [ ]:
#change the datatype of some categorical variables
df['weft_count']= df['weft_count'].apply(lambda x : str(int(x)))
df['epi']= df['epi'].apply(lambda x: str(x))
df['ppi']= df['ppi'].apply(lambda x: str(x))

In [ ]:
#dependant and independent variables
X = df.copy()
y = X.pop('Rejection')

In [ ]:
#train test split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=0)

In [ ]:
#implementing one hot encoding
ohe = OneHotEncoder(handle_unknown='ignore')
ohe_cols = [i for i in X_train.columns if len(X_train[i].value_counts())<30]
non_ohe_cols = [j for j in X_train.columns if j not in ohe_cols]
ohe_df = pd.DataFrame(ohe.fit_transform(X_train[ohe_cols]).toarray())
ohe_df.index = X_train.index
X_train_ohe = pd.concat([X_train[non_ohe_cols], ohe_df], axis=1)
ohe_df_test = pd.DataFrame(ohe.transform(X_test[ohe_cols]).toarray())
ohe_df_test.index = X_test.index
X_test_ohe = pd.concat([X_test[non_ohe_cols], ohe_df_test], axis=1)

In [ ]:
#checming the shape of train and test set
X_test_ohe.shape, X_train_ohe.shape, X_train.shape

((4402, 64), (17608, 64), (17608, 10))

In [ ]:
#changing the datatype of column name as One hot encoding can create column names with number
X_train_ohe.columns= X_train_ohe.columns.astype(str)
X_test_ohe.columns = X_test_ohe.columns.astype(str)

In [ ]:
# implemting Randomized Search CV
def calculate_gridcv(model,params, X_train, y_train,X_test, y_test, cv,n_iter):
    gdcv = RandomizedSearchCV(estimator=model,param_distributions=params, n_iter= n_iter, cv=cv, n_jobs=-1)
    gdcv.fit(X_train, y_train)
    best_model= gdcv.best_estimator_
    y_pred = gdcv.predict(X_test)
    score = r2_score(y_test, y_pred)
    best_estimator = gdcv.best_estimator_
    return [best_model, y_pred, score, best_estimator]


model_to_train = [RandomForestRegressor(), lightgbm.LGBMRegressor(), XGBRegressor(),
                  KNeighborsRegressor(), DecisionTreeRegressor(),
                  ]

grid_parameters = [
    {#random forest
        'n_estimators': [100, 200, 300, 400, 500],
'criterion': ['squared_error', 'absolute_error', 'friedman_mse', 'poisson'],
          'max_features':['sqrt', 'log2', None]},

  #Lightgbm
 {'boosting_type': ['gbdt','dart' ], 'num_leaves': [31, 50,60],
           'learning_rate':[0.1, 0.01],'n_estimators': [100, 200, 300],
           'importance_type': ['split', 'gain']}

{
    #XGboost
    'n_estimators':[50,100,200, 300,400 ],  'learning_rate':[0.1, 0.3, 0.5, 0.8] ,
          'booster': ['gbtree', 'gblinear', 'dart'],'max_depth': [1, 2,3],
         'sampling_method': ['uniform', 'gradient_based' ], 'gamma': [0, 0.01,0.5, 0.9 ], 'grow_policy' :['depthwise', 'lossguide']  },

    {#KNeighborsRegressor
        'n_neighbors': [5,10, 15,20], 'weights':['uniform', 'distance'], 'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute']},

    { #Decison Tree
    'criterion':['squared_error'], 'splitter': ['best', 'random'],
    'min_samples_leaf': [1,2], 'min_weight_fraction_leaf': [0.1,0.3,0.5], 'max_features': ['auto', 'sqrt', 'log2']
}]


model_pred_scores =[]
for i, model in enumerate(model_to_train):
    params = grid_parameters[i]
    results = calculate_gridcv(model, params, X_train_ohe,y_train, X_test_ohe, y_test, 5, 8)
    model_pred_scores.append(results)
for result in model_pred_scores:
    print(f'Model: {type(result[0]).__name__}, Score: {result[2]}, best_model:{result[3]}')